In [54]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [59]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [60]:
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.linear_model import Lasso, LassoCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from xgboost import XGBRegressor

In [61]:
df = pd.read_csv('new_data.csv')
df2 = df.drop(columns = ['Year/Month', 'Unnamed: 0'])
df3 = pd.get_dummies(df2, columns = ['Country_Code', 'month'])
df3['lag_fatal_1'] = df3['fatalities'].shift(4) # Add three month lag - Found in Tuning Notebook
df3['lag_fatal_2'] = df3['fatalities'].shift(8) # Add three month lag - Found in Tuning Notebook
df3['lag_fatal_3'] = df3['fatalities'].shift(12) # Add three month lag - Found in Tuning Notebook
df3.lag_fatal_1[0:4] = [68,0,15,47] # Add missing values (33,0,97,172,52,0,6,73,) <-- Add to front for more months
df3.lag_fatal_2[0:8] = [52,0,6,73,68,0,15,47]
df3.lag_fatal_3[0:12] = [33,0,97,172,52,0,6,73,68,0,15,47]

X = df3.drop(columns = 'fatalities')
y = df3.fatalities
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=.2, random_state=42)
#X_train = X[:-36]
#X_test = X[-36:-4]
#y_train = y[:-36]
#y_test = y[-36:-4]

X_pred = X[-4:]
y_pred = y[-4:]
X = X[:-4]
y = y[:-4]

C:\Users\ssq10\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\ssq10\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\ssq10\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [7]:
pipe_models = [('scaler', MinMaxScaler()), ('LR', LinearRegression()),
              ('Rdg', Ridge()),('Lasso',Lasso()),
              ('Tree', DecisionTreeRegressor()),('SVR',SVR(kernel='rbf')),
              ('NB', BayesianRidge()),('RF', RandomForestRegressor()),
              ('Ada', AdaBoostRegressor),('XGB', XGBRegressor())]

In [68]:
param_grid = {
              'RF__n_estimators': [60,70,80,90,100],
              'RF__max_features': [16],
              'RF__max_depth': [6]
}

In [69]:
pipe = Pipeline([('scaler', MinMaxScaler()), ('RF', RandomForestRegressor())
                ])
grid = GridSearchCV(pipe,param_grid=param_grid, cv=5, iid=False)
grid.fit(X,y)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        MinMaxScaler(copy=True,
                                                     feature_range=(0, 1))),
                                       ('RF',
                                        RandomForestRegressor(bootstrap=True,
                                                              criterion='mse',
                                                              max_depth=None,
                                                              max_features='auto',
                                                              max_leaf_nodes=None,
                                                              min_impurity_decrease=0.0,
                                                              min_impurity_split=None,
                                                              min_samples_leaf=1,
       

In [70]:
grid.best_params_

{'RF__max_depth': 6, 'RF__max_features': 16, 'RF__n_estimators': 80}

In [ ]:
'SVR__C':[.1,1,10,20],
             'SVR__gamma':[.1,1,10,20],
             'Rdg__alpha':10**np.linspace(10,-2,100),
             'Lasso__alpha':10**np.linspace(10,-2,100),
             'Tree__max_features': [2,4,6],
             'Tree__max_depth': [5, 7, 10, 15],
             'Ada__n_estimators': [50,100,200],
             'Ada__learning_rate': [.1,.5,1,5],
             'XGB__n_estimators': [50,100,200],
             'XGB__learning_rate': [.1,.5,1,5],
             'XGB__max_depth': [2, 3, 5, 10],
             'XGB__early_stopping_rounds': [10,30,50,100]

In [ ]:
RandomForestRegressor(n_estimators = 135,max_features = 16, max_depth = 6)
AdaBoostRegressor(n_estimators=100, learning_rate=.05)
XGBRegressor(n_estimators=30, learning_rate=.06)
Ridge(alpha=24.77)
Lasso(alpha=8.11)
SVR(kernel='rbf',gamma=.09)
DecisionTreeRegressor(max_depth=5,max_features=4)